# Deliverable 2. Create a Customer Travel Destinations Map.
----
1. Create a folder called `Vacation_Search` to save all the files related with this deliverable.

2. Download the `Vacation_Search_starter_code.ipynb` Jupyter notebook,save it into your `Vacation_Search folder`, and rename it as `Vacation_Search.ipynb`.

3. In the `Vacation_Search.ipynb` file, ensure that the dependencies and the Google API key is imported correctly.

4. From the `Weather_Database` folder you created in the "Deliverable 1," import the `WeatherPy_Database.csv` file as a Pandas DataFrame named `city_data_df`.

5. Write two input statements that prompt the user to enter their minimum and maximum temperature criteria for their vacation.

6. Create a new Pandas DataFrame by using the `loc` Pandas method to filter the `city_data_df` DataFrame for temperature criteria  collected. Name the DataFrame as `preferred_cities_df`.

7. Create a new Pandas DataFrame named `clean_travel_cities` by using the Pandas `dropna` function on the `preferred_cities_df` to drop any empty rows.

8. Use the `copy` Pandas function to create a new DataFrame, called `hotel_df`, by copying the following columns from the `clean_travel_cities` DataFrame: "City", "Country", "Max Temp", "Current Description", "Lat", "Lng".

9. Add a new empty column named `Hotel Name` to the `hotel_df` DataFrame.

10. Review the hotel search parameters provided. These parameters are the same we used in this module; you'll use them to search for a hotel for each city.

11. Use a for loop to iterate through the `hotel_df` DataFrame, retrieve the latitude and longitude of each city to find the nearest hotel based on the search parameters provided, then add the hotel name to the `hotel_df` DataFrame. If a hotel isn't found, skip to the next city.

12. Drop any rows in the `hotel_df` DataFrame where a hotel name is not found and store the resulting data into a new DataFrame named `clean_hotel_df`.

13. Create an CSV file to store the `clean_hotel_df` DataFrame as `WeatherPy_vacation.csv` in the Vacation_Search folder.

14. Review the formatting template provided that you'll use to add an information box to each marker in the map. In the pop-up for each city you'll add:

    1. The city name.

    2. The country code.

    3. The weather description and maximum temperature for the city.

15. Review the provided list comprehension code to retrieve the city data from each row, which will then be added to the formatting template and saved in the `hotel_info` list.

16. Use the provided code snippet to retrieve the latitude and longitude from each row and store them in a new DataFrame called `locations`.

17. Refactor your previous marker layer map code to create a marker layer map that will have pop-up markers for each city on the map.

18. Take a screenshot of your map and save it to the `Vacation_Search` folder as `WeatherPy_vacation_map.png`.
---

## Ensure that the dependencies and the Google API key is imported correctly.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import Google API key
from config import g_key

# Configure gmaps
gmaps.configure(api_key=g_key)

## From the `Weather_Database` folder you created in the "Deliverable 1," import the `WeatherPy_Database.csv` file as a Pandas DataFrame.

In [3]:
# Set the file path to import the WeatherPy_database.csv file
file_path = "../Weather_Database/WeatherPy_database.csv"

# Load the CSV file into a Pandas DataFrame
city_data_df = pd.read_csv(file_path)

# Display sample data
city_data_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Muros,ES,42.7762,-9.0603,50.09,94,0,7.54,clear sky
1,1,Albany,US,42.6001,-73.9662,65.68,94,100,4.00,heavy intensity rain
2,2,Qaanaaq,GL,77.4840,-69.3632,-4.25,65,0,7.67,clear sky
3,3,Avarua,CK,-21.2078,-159.7750,75.25,69,20,11.50,few clouds
4,4,Avera,US,33.1940,-82.5271,67.51,93,100,14.23,overcast clouds
5,5,Ranong,TH,9.9658,98.6348,76.93,88,99,3.89,overcast clouds
6,6,Mar Del Plata,AR,-38.0023,-57.5575,64.83,93,75,11.99,broken clouds
7,7,Santa Cruz,BO,-17.8000,-63.1667,80.06,65,53,9.22,broken clouds
8,8,Longyearbyen,SJ,78.2186,15.6401,17.44,79,0,11.50,clear sky
9,9,Banepa,NP,27.6298,85.5214,48.52,93,20,2.30,mist


## Write two input statements that prompt the user to enter their minimum and maximum temperature criteria for their vacation.

In [4]:
# Prompt the user to enter the minimum temperature criteria
min_temp = float(input("What is the minimum temperature you would like for your trip? "))

# Prompt the user to enter the maximum temperature criteria
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 80
What is the maximum temperature you would like for your trip? 95


## Create a new Pandas DataFrame by using the `loc` Pandas method to filter the `city_data_df` DataFrame for temperature criteria  collected.

In [11]:
# Filter the city_data_df DataFrame to find the cities that fit the criteria using the loc Pandas function
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] >= min_temp) & 
                                      (city_data_df["Max Temp"] <= max_temp)] 
                                    
# Display sample data
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
7,7,Santa Cruz,BO,-17.8000,-63.1667,80.06,65,53,9.22,broken clouds
13,13,San Patricio,US,28.0170,-97.5169,80.46,86,99,20.94,overcast clouds
27,27,Vaini,TO,-21.2000,-175.2000,84.36,79,20,9.22,few clouds
54,54,Hithadhoo,MV,-0.6000,73.0833,81.12,78,100,13.98,overcast clouds
57,57,Gold Coast,AU,-28.0000,153.4333,83.89,58,6,18.41,clear sky
63,63,Alofi,NU,-19.0595,-169.9187,80.49,74,75,12.66,broken clouds
72,72,Labuhan,ID,-6.8844,112.2051,82.44,75,100,5.30,moderate rain
75,75,Hilo,US,19.7297,-155.0900,80.33,66,20,12.66,few clouds
95,95,Port Hedland,AU,-20.3167,118.5667,80.65,47,12,8.05,few clouds
120,120,Santa Rosa,PH,14.3122,121.1114,84.18,80,68,8.21,broken clouds


## Create a new Pandas DataFrame named `clean_travel_cities` by using the Pandas `dropna` function on the `preferred_cities_df` to drop any empty rows.

In [13]:
# Drop any empty rows in the preferred_cities_df DataFrame and create a new DataFrame.
clean_travel_cities = preferred_cities_df.dropna()

# Display sample data
clean_travel_cities.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
7,7,Santa Cruz,BO,-17.8000,-63.1667,80.06,65,53,9.22,broken clouds
13,13,San Patricio,US,28.0170,-97.5169,80.46,86,99,20.94,overcast clouds
27,27,Vaini,TO,-21.2000,-175.2000,84.36,79,20,9.22,few clouds
54,54,Hithadhoo,MV,-0.6000,73.0833,81.12,78,100,13.98,overcast clouds
57,57,Gold Coast,AU,-28.0000,153.4333,83.89,58,6,18.41,clear sky
63,63,Alofi,NU,-19.0595,-169.9187,80.49,74,75,12.66,broken clouds
72,72,Labuhan,ID,-6.8844,112.2051,82.44,75,100,5.30,moderate rain
75,75,Hilo,US,19.7297,-155.0900,80.33,66,20,12.66,few clouds
95,95,Port Hedland,AU,-20.3167,118.5667,80.65,47,12,8.05,few clouds
120,120,Santa Rosa,PH,14.3122,121.1114,84.18,80,68,8.21,broken clouds


## Use the `copy` Pandas function to create a new DataFrame, called `hotel_df`, by copying the following columns from the `clean_travel_cities` DataFrame: "City", "Country", "Max Temp", "Current Description", "Lat", "Lng".

In [14]:
# Create DataFrame called hotel_df by copying some columns from the clean_travel_cities DataFrame.
hotel_df = clean_travel_cities[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# Display sample data
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng
7,Santa Cruz,BO,80.06,broken clouds,-17.8000,-63.1667
13,San Patricio,US,80.46,overcast clouds,28.0170,-97.5169
27,Vaini,TO,84.36,few clouds,-21.2000,-175.2000
54,Hithadhoo,MV,81.12,overcast clouds,-0.6000,73.0833
57,Gold Coast,AU,83.89,clear sky,-28.0000,153.4333
63,Alofi,NU,80.49,broken clouds,-19.0595,-169.9187
72,Labuhan,ID,82.44,moderate rain,-6.8844,112.2051
75,Hilo,US,80.33,few clouds,19.7297,-155.0900
95,Port Hedland,AU,80.65,few clouds,-20.3167,118.5667
120,Santa Rosa,PH,84.18,broken clouds,14.3122,121.1114


## Add a new empty column named `Hotel Name` to the `hotel_df` DataFrame.

In [16]:
# Add a new empty column, "Hotel Name", to the hotel_df DataFrame
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
7,Santa Cruz,BO,80.06,broken clouds,-17.8000,-63.1667,
13,San Patricio,US,80.46,overcast clouds,28.0170,-97.5169,
27,Vaini,TO,84.36,few clouds,-21.2000,-175.2000,
54,Hithadhoo,MV,81.12,overcast clouds,-0.6000,73.0833,
57,Gold Coast,AU,83.89,clear sky,-28.0000,153.4333,
63,Alofi,NU,80.49,broken clouds,-19.0595,-169.9187,
72,Labuhan,ID,82.44,moderate rain,-6.8844,112.2051,
75,Hilo,US,80.33,few clouds,19.7297,-155.0900,
95,Port Hedland,AU,80.65,few clouds,-20.3167,118.5667,
120,Santa Rosa,PH,84.18,broken clouds,14.3122,121.1114,


## Review the hotel search parameters provided. These parameters are the same we used in this module; you'll use them to search for a hotel for each city.

In [17]:
# Review the parameters to search for a hotel
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}        

## Use a for loop to iterate through the `hotel_df` DataFrame, retrieve the latitude and longitude of each city to find the nearest hotel based on the search parameters provided, then add the hotel name to the `hotel_df` DataFrame. If a hotel isn't found, skip to the next city.

In [19]:
# Iterate through the hotel DataFrame
print("Initiating Hotel Search")
print("-------------------------")
for index, row in hotel_df.iterrows():
    # Fetch latitude and longitude from the DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add the latitude and longitude as parameters to the params dictionary
    params["location"] = f"{lat},{lng}"
    
    # Set up the base URL for the Google Directions API to get JSON data
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # Make an API request and retrieve the JSON data from the hotel search
    hotels = requests.get(base_url, params=params).json()
    
    # Get the first hotel from the results and store the name, if a hotel isn't found skip the city
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
print("Hotel Search Complete")
print("-------------------------")

Initiating Hotel Search
-------------------------
Hotel not found... skipping.
Hotel not found... skipping.
Hotel Search Complete
-------------------------


In [21]:
# Display sample data
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
7,Santa Cruz,BO,80.06,broken clouds,-17.8000,-63.1667,Aviador Hotel Boutique
13,San Patricio,US,80.46,overcast clouds,28.0170,-97.5169,"Motel 6 Sinton, TX"
27,Vaini,TO,84.36,few clouds,-21.2000,-175.2000,Keleti Beach Resort
54,Hithadhoo,MV,81.12,overcast clouds,-0.6000,73.0833,Scoop Guest House
57,Gold Coast,AU,83.89,clear sky,-28.0000,153.4333,Mantra Towers of Chevron Surfers Paradise
63,Alofi,NU,80.49,broken clouds,-19.0595,-169.9187,Taloa Heights
72,Labuhan,ID,82.44,moderate rain,-6.8844,112.2051,DoubleFAG
75,Hilo,US,80.33,few clouds,19.7297,-155.0900,Hilo Hawaiian Hotel
95,Port Hedland,AU,80.65,few clouds,-20.3167,118.5667,The Esplanade Hotel
120,Santa Rosa,PH,84.18,broken clouds,14.3122,121.1114,Erlyn’s Sta Rosa City Home


## Drop any rows in the `hotel_df` DataFrame where a hotel name is not found and store the resulting data into a new DataFrame named `clean_hotel_df`.

In [24]:
# Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.loc[hotel_df["Hotel Name"] != ""]

# Display sample data
clean_hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
7,Santa Cruz,BO,80.06,broken clouds,-17.8000,-63.1667,Aviador Hotel Boutique
13,San Patricio,US,80.46,overcast clouds,28.0170,-97.5169,"Motel 6 Sinton, TX"
27,Vaini,TO,84.36,few clouds,-21.2000,-175.2000,Keleti Beach Resort
54,Hithadhoo,MV,81.12,overcast clouds,-0.6000,73.0833,Scoop Guest House
57,Gold Coast,AU,83.89,clear sky,-28.0000,153.4333,Mantra Towers of Chevron Surfers Paradise
63,Alofi,NU,80.49,broken clouds,-19.0595,-169.9187,Taloa Heights
72,Labuhan,ID,82.44,moderate rain,-6.8844,112.2051,DoubleFAG
75,Hilo,US,80.33,few clouds,19.7297,-155.0900,Hilo Hawaiian Hotel
95,Port Hedland,AU,80.65,few clouds,-20.3167,118.5667,The Esplanade Hotel
120,Santa Rosa,PH,84.18,broken clouds,14.3122,121.1114,Erlyn’s Sta Rosa City Home


**Note:** If the resulting DataFrame is empty, uncomment the following code to load sample data into the `clean_hotel_df`.

In [ ]:
# In case of an empty DataFrame, load the sample data provided
# clean_hotel_df = pd.read_csv("WeatherPy_vacation-backup.csv")
# clean_hotel_df.head(10)

## Create an CSV file to store the `clean_hotel_df` DataFrame as `WeatherPy_vacation.csv` in the Vacation_Search folder.

In [25]:
# Set the file name.
output_data_file = "WeatherPy_vacation.csv"

# Create a CSV file by using the clean_hotel_df DataFrame
clean_hotel_df.to_csv(output_data_file)

### Review the formatting template provided that you'll use to add an information box to each marker in the map.

In the pop-up for each city you'll add: 

1. The city name

2. The country code

3. The weather description and maximum temperature for the city

In [26]:
# Review the formatting template provided
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

## Review the provided list comprehension code to retrieve the city data from each row, which will then be added to the formatting template and saved in the `hotel_info` list.

In [27]:
# Get the data from each row in the clean_hotel_df DataFrame, add it to the formatting template, and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# Display sample data
hotel_info[:10]

['\n<dl>\n<dt>Hotel Name</dt><dd>Aviador Hotel Boutique</dd>\n<dt>City</dt><dd>Santa Cruz</dd>\n<dt>Country</dt><dd>BO</dd>\n<dt>Current Weather</dt><dd>broken clouds and 80.06 °F</dd>\n</dl>\n',
 '\n<dl>\n<dt>Hotel Name</dt><dd>Motel 6 Sinton, TX</dd>\n<dt>City</dt><dd>San Patricio</dd>\n<dt>Country</dt><dd>US</dd>\n<dt>Current Weather</dt><dd>overcast clouds and 80.46 °F</dd>\n</dl>\n',
 '\n<dl>\n<dt>Hotel Name</dt><dd>Keleti Beach Resort</dd>\n<dt>City</dt><dd>Vaini</dd>\n<dt>Country</dt><dd>TO</dd>\n<dt>Current Weather</dt><dd>few clouds and 84.36 °F</dd>\n</dl>\n',
 '\n<dl>\n<dt>Hotel Name</dt><dd>Scoop Guest House</dd>\n<dt>City</dt><dd>Hithadhoo</dd>\n<dt>Country</dt><dd>MV</dd>\n<dt>Current Weather</dt><dd>overcast clouds and 81.12 °F</dd>\n</dl>\n',
 '\n<dl>\n<dt>Hotel Name</dt><dd>Mantra Towers of Chevron Surfers Paradise</dd>\n<dt>City</dt><dd>Gold Coast</dd>\n<dt>Country</dt><dd>AU</dd>\n<dt>Current Weather</dt><dd>clear sky and 83.89 °F</dd>\n</dl>\n',
 '\n<dl>\n<dt>Hotel 

## Use the provided code snippet to retrieve the latitude and longitude from each row and store them in a new DataFrame called `locations`.

In [28]:
# Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

# Display sample data
locations.head(10)

,Lat,Lng
7,-17.8000,-63.1667
13,28.0170,-97.5169
27,-21.2000,-175.2000
54,-0.6000,73.0833
57,-28.0000,153.4333
63,-19.0595,-169.9187
72,-6.8844,112.2051
75,19.7297,-155.0900
95,-20.3167,118.5667
120,14.3122,121.1114


## Refactor your previous marker layer map code to create a marker layer map that will have pop-up markers for each city on the map.

In [33]:
# Add a marker layer for each city to the map. 
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

# Create a figure to add the Google map as a layer
fig = gmaps.figure(center=(13.0, 37.0), zoom_level=2)
fig.add_layer(marker_layer)

# Display the figure containing the map
fig


Figure(layout=FigureLayout(height='420px'))